# Synopsis

Convert [General Inquirer lexicon](http://www.wjh.harvard.edu/~inquirer/spreadsheet_guide.htm) into usable form. Download [the spreadsheet](http://www.wjh.harvard.edu/~inquirer/inquirerbasic.xls).

# Libraries

In [2]:
import pandas as pd
import numpy as np

# Pragmas

In [3]:
%matplotlib inline

# Process

## Import Excel file 

It has 182 ontology/sentiment columns

In [4]:
df = pd.read_excel('lexicons/inquirerbasic.xls', false_values=None)

In [7]:
df.sample(10)

,Entry,Source,Positiv,Negativ,Pstv,Affil,Ngtv,Hostile,Strong,Power,...,Anomie,NegAff,PosAff,SureLw,If,NotLw,TimeSpc,FormLw,Othtags,Defined
7259,OUT,H4Lvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,TimeSpc,NaN,PREP LY,"| prep-adv: Outside of, away from, empty--into..."
3477,EMPIRE,H4Lvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Noun,|
10216,SURGE,H4,Positiv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Noun,|
151,ADDICTION,H4,NaN,Negativ,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Noun,|
6307,MAINTAIN#2,H4Lvd,NaN,NaN,NaN,NaN,NaN,NaN,Strong,Power,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUPV,"| 14% verb: To assert, contend"
8286,RAPID#1,H4Lvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Modif,"| 39% adj: Quick, fast"
11567,WIDE#2,H4Lvd,NaN,NaN,NaN,NaN,NaN,NaN,Strong,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,TimeSpc,NaN,LY,"| 14% adv: ""Widely""--broadly"
7344,OWNER,H4Lvd,NaN,NaN,NaN,NaN,NaN,NaN,Strong,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Noun,"| noun: One who owns, a proprietor"
183,ADMIRAL#2,Lvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Noun HU,NaN
7106,OCCASION#2,H4Lvd,NaN,NaN,NaN,NaN,NaN,NaN,Strong,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUPV,"| 7% verb: To give cause for, to bring about"


## Handle variant terms 

In [33]:
df['split'] = df.Entry.str.split(r'[#_]+')

In [34]:
# Function to add 1 to terms without variants
def add_1(x):
    try:
        if len(x) == 1:
            x = x + [1]
    except:
        print(x)
        x = 0
    return x

In [35]:
df['split'] = df['split'].apply(add_1)
df['term_str'] = df['split'].apply(lambda x: x[0].lower(), 1)
df['term_var'] = df['split'].apply(lambda x: x[1], 1)
df = df.set_index(['term_str', 'term_var'])

In [36]:
df.head()

,,Entry,Source,Positiv,Negativ,Pstv,Affil,Ngtv,Hostile,Strong,Power,...,PosAff,SureLw,If,NotLw,TimeSpc,FormLw,Othtags,Defined,split,polarity
term_str,term_var,,,,,,,,,,,,,,,,,,,,,
a,1,A,H4Lvd,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,DET ART,| article: Indefinite singular article--some o...,"[A, 1]",0.0
abandon,1,ABANDON,H4Lvd,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,SUPV,|,"[ABANDON, 1]",-1.0
abandonment,1,ABANDONMENT,H4,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,Noun,|,"[ABANDONMENT, 1]",-1.0
abate,1,ABATE,H4Lvd,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,SUPV,|,"[ABATE, 1]",-1.0
abatement,1,ABATEMENT,Lvd,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,Noun,NaN,"[ABATEMENT, 1]",0.0


## Convert column values to numbers

In [41]:
try:
    df.loc[df.Positiv == 'Positiv', 'polarity'] = 1
    df.loc[df.Negativ == 'Negativ', 'polarity'] = -1
    df['polarity'] = df['polarity'].fillna(0)
except TypeError as e:
    print(e, "--  This means you are re-running the script with values that have already been set.")

invalid type comparison --  This means you are re-running the script with values that have already been set.


In [42]:
df.polarity.sample(10)

term_str      term_var
immediate     1           0.0
undergo       1           0.0
conservative  1           0.0
stuff         2           0.0
good          2           0.0
tense         1          -1.0
clamorous     1          -1.0
accelerate    1           0.0
least         3           0.0
rule          4           0.0
Name: polarity, dtype: float64

In [43]:
VALCOLS = df.columns[2:-4]

In [44]:
for col in VALCOLS:
    df[col] = df[col].fillna(0).astype('bool').astype('int')

In [62]:
df.loc['love', VALCOLS].stack().sort_values(ascending=False)

term_var         
1         Positiv    1
5         Active     1
2         AffOth     1
1         AffTot     1
          AffGain    1
5         SocRel     1
3         AffGain    1
          AffTot     1
1         SV         1
5         Strong     1
4         AffGain    1
5         Affil      1
4         Positiv    1
          Pstv       1
          Affil      1
          Passive    1
          Pleasur    1
          EMOT       1
2         Positiv    1
          Pstv       1
          Affil      1
          Passive    1
6         SocRel     1
          Affil      1
3         Positiv    1
          Pstv       1
          Affil      1
          Passive    1
          Pleasur    1
          EMOT       1
                    ..
5         RcGain     0
          RcEnds     0
          Self       0
          RcTot      0
          RspGain    0
          RspLoss    0
          RspOth     0
          RspTot     0
          AffLoss    0
          AffPt      0
          PowPt      0
          PowAuP

In [47]:
# # Handle terms with multiple entries
# VAR_COUNTS = df.reset_index().groupby(['term_str']).term_var.count()\
#     .to_frame().rename(columns={'term_var':'n_vars'})
# SINGLES = VAR_COUNTS == 1
# SINGLES.head()
# df.loc['a'].T
# df.loc[SINGLES.n_vars.values]

## Take and combine only sentiment 

In [48]:
sents = df.query("Positiv > 0 | Negativ > 0").copy()

In [54]:
sents['sentiment'] =  sents.Positiv  - sents.Negativ

In [55]:
sents['sentiment'].sample(5)

term_str       term_var
fine           5           1
accession      1           1
augment        1           1
controversial  1          -1
exception      2          -1
Name: sentiment, dtype: int64

### Take average of variants

In [58]:
GI = sents.groupby('term_str').sentiment.mean()

### Snap values to -1, 0, or 1

In [59]:
GI = np.sign(GI).astype('int').to_frame()

In [60]:
GI.shape

In [8]:
GI.head()

NameError: name 'GI' is not defined

# Save

In [29]:
GI.to_csv('lexicons/gi.csv')